In [1]:
import matplotlib
matplotlib.use('Agg')

In [2]:
import skimage
import skimage.feature
import skimage.viewer
from skimage.viewer.widgets import Slider, Button
from skimage.viewer.qt import QtWidgets, QtGui, Qt, Signal
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from matplotlib import pyplot as plt
from tkinter import *
from skimage.viewer.plugins.overlayplugin import OverlayPlugin
import matplotlib.colors as mcolors
from pandas import *
# from skimage.viewer.plugins.canny import CannyPlugin
# from skimage.viewer.canvastools import PaintTool
from skimage.viewer.canvastools.base import CanvasToolBase, ToolHandles
from matplotlib import lines
import matplotlib.colors as mcolors
from matplotlib.widgets import RectangleSelector
# LABELS_CMAP = mcolors.ListedColormap(mcolors.CSS4_COLORS)
LABELS_CMAP = mcolors.ListedColormap(['white', 'red', 'dodgerblue', 'gold',
                                      'greenyellow', 'blueviolet'])
from matplotlib import backend_bases
# mpl.rcParams['toolbar'] = 'None'
backend_bases.NavigationToolbar2.toolitems = (
        ('Home', 'Reset original view', 'home', 'home'),
        ('Back', 'Back to  previous view', 'back', 'back'),
        ('Forward', 'Forward to next view', 'forward', 'forward'),
        (None, None, None, None),
        ('Zoom', 'Zoom to rectangle', 'zoom_to_rect', 'zoom'),
        (None, None, None, None),
        ('Save', 'Save the figure', 'filesave', 'save_figure'),
      )

from skimage.util.dtype import dtype_range
from skimage.exposure import rescale_intensity
from skimage.viewer.qt import QtWidgets, QtGui, Qt, Signal
from skimage.viewer.utils import (dialogs, init_qtapp, figimage, start_qtapp,
                     update_axes_image)
from skimage.viewer.utils.canvas import BlitManager, EventManager
from skimage.viewer.plugins.base import Plugin


In [3]:
arraysegmented = []
slices = 0
pixelarray = []
labels = 1
firstimage = []
pixelvalue = [0, 50, 100, 150, 200, 250]


filename = "../../datasets/Train_Dicom/00-07-40-63(ada garis).bin"
outfile = "../../datasets/Train_Dicom/00-07-40-63(ada garis).bin"
f = open(filename, "rb")
#np.frombuffer(f.read(52), dtype='f')[0:3]

data = np.frombuffer(f.read(), dtype=np.uint8)
plt.imshow(data)
plt.show()


# data = np.frombuffer(f.read(), dtype=np.uint8).reshape((256, 256))
# #data2 = np.frombuffer(open(outfile, 'rb').read(), dtype=np.uint8).reshape(256, 256, 256)
# new_data = np.zeros((256, 256, 256), dtype='uint8')
# # Mask Sagital ke array baru
# for i in range(256):
#     new_data[:,:,i] = data[:,:,i]
# # Mask Coronal ke array baru
# for i in range(256):
#     new_data[:,i,:] = data[:,i,:]
# # Mask Axial ke array baru
# for i in range(256):
#     new_data[i,:,:] = data[i,:,:]

TypeError: Invalid shape (16777268,) for image data

In [ ]:
class CenteredWindow(object):
    """Window that create slices numpy arrays over 2D windows.
    Examples
    --------
    >>> a = np.arange(16).reshape(4, 4)
    >>> w = CenteredWindow(1, a.shape)
    >>> a[w.at(1, 1)]
    array([[ 0,  1,  2],
           [ 4,  5,  6],
           [ 8,  9, 10]])
    >>> a[w.at(0, 0)]
    array([[0, 1],
           [4, 5]])
    >>> a[w.at(4, 3)]
    array([[14, 15]])
    """
    def __init__(self, radius, array_shape):
        self.radius = radius
        self.array_shape = array_shape

    def at(self, row, col, button):
        h, w = self.array_shape
        r = round(self.radius)
        # Note: the int() cast is necessary because row and col are np.float64,
        # which does not get cast by round(), unlike a normal Python float:
        # >>> round(4.5)
        # 4
        # >>> round(np.float64(4.5))
        # 4.0
        # >>> int(round(np.float64(4.5)))
        # 4
        row, col = int(round(row)), int(round(col))
        xmin = max(0, col - r)
        xmax = min(w, col + r + 1)
        ymin = max(0, row - r)
        ymax = min(h, row + r + 1)
        print(ymin, ymax, xmin, xmax)
        self.radiusPixel(ymin, ymax, xmin, xmax, button)
        
        return (slice(ymin, ymax), slice(xmin, xmax))
    
    def radiusPixel(self, ymin, ymax, xmin, xmax, button):
        for i in range(ymin, ymax):
            for j in range(xmin, xmax):
                if button == 1:
                    arraysegmented.append([slices, int(round(j, 0)), int(round(i, 0)), labels])
                else:
                    arraysegmented.append([slices, int(round(j, 0)), int(round(i, 0)), 0])
                # print(arraysegmented)

In [ ]:
#new_data[data2==150] = 150
#new_data[119:136] = data[119:136]
class PaintTool(CanvasToolBase):
    def __init__(self, manager, overlay_shape, radius=1, alpha=0.2,
                 on_move=None, on_release=None, on_enter=None,
                 rect_props=None):
        super(PaintTool, self).__init__(manager, on_move=on_move,
                                        on_enter=on_enter,
                                        on_release=on_release)

        props = dict(edgecolor='r', facecolor='0.7', alpha=0.5, animated=True)
        props.update(rect_props if rect_props is not None else {})

        self.alpha = alpha
        self.cmap = LABELS_CMAP
        self._overlay_plot = None
        self.shape = overlay_shape[:2]

        self._cursor = plt.Rectangle((0, 0), 0, 0, **props)
        self._cursor.set_visible(False)
        self.ax.add_patch(self._cursor)

        # `label` and `radius` can only be set after initializing `_cursor`
        self.label = labels
        self.radius = radius

        # Note that the order is important: Redraw cursor *after* overlay
        self.artists = [self._overlay_plot, self._cursor]
        self.manager.add_tool(self)

    @property
    def label(self):
        return self._label

    @label.setter
    def label(self, value):
        if value >= self.cmap.N:
            raise ValueError('Maximum label value = %s' % len(self.cmap - 1))
        self._label = value
        
        self._cursor.set_edgecolor(self.cmap(value))

    @property
    def radius(self):
        return self._radius

    @radius.setter
    def radius(self, r):
        self._radius = r
        self._width = 2 * r + 1
        self._cursor.set_width(self._width)
        self._cursor.set_height(self._width)
        self.window = CenteredWindow(r, self._shape)

    @property
    def overlay(self):
        return self._overlay

    @overlay.setter
    def overlay(self, image):
        global firstimage
        firstimage = image
        self._overlay = image
        # print(self._overlay_plot)
        if image is None:
            self.ax.images.remove(self._overlay_plot)
            self._overlay_plot = None
        elif self._overlay_plot is None:
            props = dict(cmap=self.cmap, alpha=self.alpha,
                          norm=mcolors.NoNorm(vmin=0, vmax=self.cmap.N),
                          animated=True)
            self._overlay_plot = self.ax.imshow(image, **props)
        else:
            self._overlay_plot.set_data(image)
        
        self.redraw()

    @property
    def shape(self):
        return self._shape

    @shape.setter
    def shape(self, shape):
        self._shape = shape
        if not self._overlay_plot is None:
            self._overlay_plot.set_extent((-0.1, shape[1] + 0.1,
                                           shape[0] + 0.1, -0.1))
            self.radius = self._radius
        self.overlay = np.zeros(shape, dtype='uint8')

    def on_key_press(self, event):
        global labels
        if event.key == 'enter':
            if labels==5:
                self.overlay = np.zeros(self.shape, dtype='uint8')
                labels = 0
                self.label = labels
                
            self.callback_on_enter(self.geometry)
            self.redraw()
            # self.remove_overlay(event)
            oldlabel = self.label
            self.label = oldlabel+1
            labels = self.label
            print(labels)
            
    def on_mouse_press(self, event):
        if event.button not in [1, 3] or not self.ax.in_axes(event):
            return
        self.update_cursor(event.xdata, event.ydata)
        self.update_overlay(event.xdata, event.ydata, event.button)
        # self.get_coordinat(event.xdata, event.ydata)

    def on_mouse_release(self, event):
        if event.button != 1:
            return
        self.callback_on_release(self.geometry)

    def on_move(self, event):
        if not self.ax.in_axes(event):
            self._cursor.set_visible(False)
            self.redraw() # make sure cursor is not visible
            return
        self._cursor.set_visible(True)
        self.update_cursor(event.xdata, event.ydata)
        # self.on_mouse_press(event)
        
        if event.button not in [1, 3]:
            self.redraw() # update cursor position
            return
        self.update_overlay(event.xdata, event.ydata, event.button)
        self.callback_on_move(self.geometry)

    def update_overlay(self, x, y, button):
        overlay = self.overlay
        if button == 1:
            overlay[self.window.at(y, x, button)] = self.label
        else:
            overlay[self.window.at(y, x, button)] = 0
        # Note that overlay calls `redraw`
        self.overlay = overlay

    def update_cursor(self, x, y):
        x = x - self.radius - 1
        y = y - self.radius - 1
        self._cursor.set_xy((x, y))

    def get_coordinat(self, x, y):
        # print(int(round(x, 0)), int(round(y, 0)), self.label)
        print("hey")
        arraysegmented.append([slices, int(round(x, 0)), int(round(y, 0)), self.label])
        
    def remove_overlay(self, image):
        self.overlay()
        

    def on_scroll(self, event):
        if event.button == 'up':
            r = self._radius
            r = r + 1 if r + 1 < 10 else 10
            self.radius = r
            # print('going up bcs im not feeling down')
        else:
            r = self._radius
            r = r - 1 if r - 1 > 1 else 1
            self.radius = r
            # print('going down bcs nothing is up')
            
        self.redraw()
        
    @property
    def geometry(self):
        return self.overlay